In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc, classification_report

import tensorflow as tf

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score




from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score




#### Exploring final df

In [2]:
# Load the saved dataset
df = pd.read_csv('processed_data/merged_df.csv')

print(f"Dataset loaded: {df.shape}")
print(f"Columns: {list(df.columns)}")

Dataset loaded: (18140, 44)
Columns: ['gross_FU', 'gross_SC', 'net_FU', 'net_SC', 'time_first_sc_to_first_net_fu', 'electricitybill', 'heatingbill', 'netcontractsigned', 'selfipa_done', 'zipregion_missing', 'evaluationtime_missing', 'desiredinstallationend_missing', 'electricitybill_missing', 'heatingbill_missing', 'mktgparamscore_missing', 'desiredinstallationend_encoded', 'mktg_High', 'mktg_Low', 'mktg_Medium', 'region_High_Performer', 'region_Large_Good', 'region_Medium', 'region_Other', 'total_bc_attempts', 'total_bc_outcomes', 'lead_to_first_bc_days', 'bc_duration_days', 'lead_to_sc1_days', 'sc1_schedule_to_appointment_days', 'bc_frequency', 'positive_outcomes_count', 'negative_outcomes_count', 'noshow_outcomes_count', 'positive_outcome_ratio', 'negative_outcome_ratio', 'noshow_outcome_ratio', 'reachability_score', 'outcome_trend', 'persistence_after_negative', 'showed_up_sc1', 'engagement_score', 'efficiency_score', 'last_bc_outcome_encoded', 'first_bc_outcome_encoded']


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18140 entries, 0 to 18139
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gross_FU                          18140 non-null  int64  
 1   gross_SC                          18140 non-null  int64  
 2   net_FU                            18140 non-null  float64
 3   net_SC                            18140 non-null  float64
 4   time_first_sc_to_first_net_fu     18140 non-null  float64
 5   electricitybill                   18140 non-null  float64
 6   heatingbill                       18140 non-null  float64
 7   netcontractsigned                 18140 non-null  float64
 8   selfipa_done                      18140 non-null  int64  
 9   zipregion_missing                 18140 non-null  int64  
 10  evaluationtime_missing            18140 non-null  int64  
 11  desiredinstallationend_missing    18140 non-null  int64  
 12  elec

In [5]:
value_counts_with_nan = df['showed_up_sc1'].value_counts(dropna=False)
print("Value counts (including NaN):\n", value_counts_with_nan)


Value counts (including NaN):
 showed_up_sc1
0.0    14579
1.0     3561
Name: count, dtype: int64


### Data Understanding of this new df

In [19]:
# Check the target variable
print("=== TARGET VARIABLE ANALYSIS ===")
print(f"Total samples: {len(df)}")
print(f"Conversion rate: {df['netcontractsigned'].mean():.3f}")
print(f"Converted: {df['netcontractsigned'].sum():.0f}")
print(f"Not converted: {(len(df) - df['netcontractsigned'].sum()):.0f}")

=== TARGET VARIABLE ANALYSIS ===
Total samples: 18140
Conversion rate: 0.023
Converted: 426
Not converted: 17714


In [20]:
# Separate features from target
target_column = 'netcontractsigned'
X = df.drop(target_column, axis=1)
y = df[target_column]

print("=== DATA PREPARATION ===")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Feature columns: {X.columns.tolist()[:10]}...")  # Show first 10 columns
print(f"Any missing values in features: {X.isnull().sum().sum()}")
print(f"Any missing values in target: {y.isnull().sum()}")

=== DATA PREPARATION ===
Features shape: (18140, 43)
Target shape: (18140,)
Feature columns: ['gross_FU', 'gross_SC', 'net_FU', 'net_SC', 'time_first_sc_to_first_net_fu', 'electricitybill', 'heatingbill', 'selfipa_done', 'zipregion_missing', 'evaluationtime_missing']...
Any missing values in features: 0
Any missing values in target: 0


#### To see any signal at all is present

In [23]:

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("=== TRAIN/TEST SPLIT ===")
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")
print(f"Train conversion rate: {y_train.mean():.3f}")
print(f"Test conversion rate: {y_test.mean():.3f}")

# Scale the features (important for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a simple logistic regression (NO imbalance handling yet)
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_scaled, y_train)

# Get predictions
y_pred_proba = lr.predict_proba(X_test_scaled)[:, 1]

# Check if there's any signal
auc_score = roc_auc_score(y_test, y_pred_proba)

print("\n=== SIGNAL TEST RESULTS ===")
print(f"AUC-ROC Score: {auc_score:.3f}")
if auc_score > 0.5:
    print("✅ GOOD NEWS: Your features contain useful signal!")
    print("We can proceed with imbalance fixing strategies.")
else:
    print("❌ BAD NEWS: No signal detected. Features need work first.")

=== TRAIN/TEST SPLIT ===
Train size: 14512
Test size: 3628
Train conversion rate: 0.023
Test conversion rate: 0.023

=== SIGNAL TEST RESULTS ===
AUC-ROC Score: 0.880
✅ GOOD NEWS: Your features contain useful signal!
We can proceed with imbalance fixing strategies.


=== TRAIN/TEST SPLIT ===
Train size: 14512
Test size: 3628
Train conversion rate: 0.023
Test conversion rate: 0.023

=== SIGNAL TEST RESULTS ===
AUC-ROC Score: 0.880
✅ GOOD NEWS: Your features contain useful signal!
We can proceed with imbalance fixing strategies.

however lets take a step further to evaluate imbalanced data

In [ ]:


# Calculate AUC-PR (more reliable for imbalanced data)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auc_pr = auc(recall, precision)

# Random baseline for comparison
random_baseline = y_test.mean()  # 0.023

print("\n=== DETAILED SIGNAL ANALYSIS ===")
print(f"AUC-ROC: {auc_score:.3f}")
print(f"AUC-PR: {auc_pr:.3f}")
print(f"Random baseline AUC-PR: {random_baseline:.3f}")
print(f"Improvement over random: {auc_pr/random_baseline:.1f}x")

# Look at actual predictions with default 0.5 threshold
y_pred = (y_pred_proba >= 0.5).astype(int)
print(f"\nWith 0.5 threshold:")
print(f"Predicted positives: {y_pred.sum()}")
print(f"Actual positives: {y_test.sum()}")

# Check top 10% of predictions
top_10_percent_threshold = np.percentile(y_pred_proba, 90)
print(f"\nTop 10% analysis:")
print(f"Threshold for top 10%: {top_10_percent_threshold:.3f}")
top_10_mask = y_pred_proba >= top_10_percent_threshold
print(f"Conversion rate in top 10%: {y_test[top_10_mask].mean():.3f}")
print(f"That's {y_test[top_10_mask].mean()/y_test.mean():.1f}x better than average!")


=== DETAILED SIGNAL ANALYSIS ===
AUC-ROC: 0.880
AUC-PR: 0.174
Random baseline AUC-PR: 0.023
Improvement over random: 7.4x

With 0.5 threshold:
Predicted positives: 2
Actual positives: 85.0

Top 10% analysis:
Threshold for top 10%: 0.052
Conversion rate in top 10%: 0.132
That's 5.6x better than average!


=== DETAILED SIGNAL ANALYSIS ===
AUC-ROC: 0.880
AUC-PR: 0.174
Random baseline AUC-PR: 0.023
Improvement over random: 7.4x

With 0.5 threshold:
Predicted positives: 2
Actual positives: 85.0

Top 10% analysis:
Threshold for top 10%: 0.052
Conversion rate in top 10%: 0.132
That's 5.6x better than average!

## Checking feature importance

In [1]:
df.info()

NameError: name 'df' is not defined